# Reading Video
> Cinelytics uses OpenCV to read video. The main function is `read_video`, where you can flexibly grab either all or different sequences of the frames of any given video.

In [1]:
#default_exp io

In [2]:
#hide
%load_ext autoreload
%autoreload 2
%matplotlib notebook

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
#export
from videoutils.utils import *

## Reading Video using OpenCV

In [4]:
#export

def read_video(fname: Union[str, cv2.VideoCapture],
               target_frames: Union[tuple, list, int, np.array, None]=None,
               apply: Callable=None) -> Union[torch.Tensor, list]:
    """Flexible video reader where you can grab frames in different ways
    and return as different dtypes.
    """
    cap    = capture(fname)
    frames = read_all_frames(cap) if target_frames is None else read_specific_frames(cap, target_frames)
    cap.release()

    if apply is not None:
        if apply == as_tensor: frames = torch.stack(lapply(frames, apply))
        else: frames = lapply(frames, apply)
    
    return frames

**Args**:
* `fname`: path to the video file, or a `cv2.VideoCapture` object
* `target_frames`: if `None`, all the frames of `fname` are read. Else, if it is a
    - `int`: returns frame at this index
    - `list`/`np.array`: returns frame at the indices of each element
    - `tuple`: <br>a tuple like `(start_idx, end_idx, stride)` where `stride` is optional and if `stride=2`, after reading the first frame, every _2nd/(stride)th_ frame is read
* `apply`: a function that transforms a `np.array` of shape `(height, width, channels)`

---

## Helper Functions

In [5]:
#export

def read_all_frames(cap: cv2.VideoCapture) -> list:
    "Read all frames from a `cv2.VideoCapture` object"
    frames=[]
    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret is True: frames.append(bgr2rgb(frame))
        else: break

    num_frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    if not len(frames) == num_frames: print(f'Only read in {num_frames} / {len(frames)}')
        
    return frames

Helper function for `read_video`

---

In [6]:
#export

def read_specific_frames(cap: cv2.VideoCapture, 
                         target_frames: Union[tuple, list, int, np.array]) -> list:
    "Read specific frames from a `cv2.VideoCapture` object"
    if   isinstance(target_frames, tuple) : frame_idxs = np.arange(*target_frames)
    elif isinstance(target_frames, list)  : frame_idxs = target_frames
    elif isinstance(target_frames, int)   : frame_idxs = [target_frames]
    elif isinstance(target_frames, np.ndarray)   : frame_idxs = target_frames 

    frames=[]
    for i in frame_idxs:
        cap.set(cv2.CAP_PROP_POS_FRAMES, i)
        ret, frame = cap.read()
        if ret==True: frames.append(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)) # return np.array
        else: break
    return frames

Helper function for `read_video`

---

In [7]:
#export

def capture(x: Union[str, cv2.VideoCapture]) -> cv2.VideoCapture:
    "Ensure `cv2.VideoCapture` works properly"
    assert isinstance(x, (str, cv2.VideoCapture)), \
    f"Expected `str` or `cv2.VideoCapture` but received {type(x)} "
    cap = cv2.VideoCapture(x) if isinstance(x, str) else x
    assert(cap.isOpened()), f'Failed to open video "{x}"'
    return cap

Helper function for `read_video`

---

In [8]:
vid = read_video('files/interstellar-waves-edit.mp4',
                 target_frames=[0,1,2,3,4,5])

In [9]:
len(vid); vid[0].shape

6

(480, 720, 3)

---

In [10]:
vid = read_video('files/interstellar-waves-edit.mp4',
                 target_frames=np.arange(100),
                 apply=as_tensor)

In [11]:
vid.shape

torch.Size([100, 3, 480, 720])

---

In [12]:
vid = read_video('files/interstellar-waves-edit.mp4',
                 target_frames=(0, 50, 2),
                 apply=as_tensor)

In [13]:
vid.shape

torch.Size([25, 3, 480, 720])

---

In [14]:
vid = read_video('files/interstellar-waves-edit.mp4',
                 target_frames=5)

In [15]:
len(vid); vid[0].shape

1

(480, 720, 3)

### Export -

In [16]:
#hide
from nbdev.export import *
notebook2script()

Converted 00_utils.ipynb.
Converted 01_io.ipynb.
